# PyTorch Tutorial: Model Deployment and Production

Training a model is only half the battle. To use it in the real world (in a mobile app, a web server, or an embedded device), you need to **deploy** it. This often means optimizing it for speed and portability.

## Learning Objectives
- Understand **TorchScript** (Tracing vs Scripting)
- Export models to **ONNX** (Open Neural Network Exchange)
- **Quantization**: Making models smaller and faster
- Measure Inference Speed (Benchmarking)

## 1. Vocabulary First

- **Inference**: Using a trained model to make predictions (no backprop).
- **Latency**: Time taken to process one input (lower is better).
- **Throughput**: Number of inputs processed per second (higher is better).
- **Serialization**: Saving a model to a file so it can be loaded anywhere (even in C++).
- **Quantization**: Reducing the precision of numbers (e.g., 32-bit float -> 8-bit integer) to save memory and compute.

In [ ]:
import torch
import torch.nn as nn
import time
import os

# Let's use a simple model for demonstration
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(10, 50)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(50, 2)

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

model = SimpleModel()
model.eval() # Important! Set to eval mode for deployment
example_input = torch.randn(1, 10)
print("Model created.")

## 2. TorchScript: Tracing vs Scripting

TorchScript allows you to serialize your models and run them in a C++ runtime (no Python needed!).

### Method A: Tracing
Runs the model with a dummy input and records the operations. Fast and easy, but fails with dynamic control flow (if/else).

```python
traced_model = torch.jit.trace(model, example_input)
traced_model.save("traced_model.pt")
```

### Method B: Scripting
Analyzes the Python source code to compile it. Handles if/else loops correctly.

```python
scripted_model = torch.jit.script(model)
scripted_model.save("scripted_model.pt")
```

In [ ]:
# Try Tracing
traced_model = torch.jit.trace(model, example_input)
print(traced_model.code) # You can see the internal representation

# Verify it works
output_orig = model(example_input)
output_traced = traced_model(example_input)
assert torch.allclose(output_orig, output_traced)
print("Tracing successful!")

## 3. ONNX Export

ONNX is a standard format supported by many frameworks (TensorFlow, PyTorch, MATLAB, etc.). It's great for deploying to edge devices or using specialized runtimes like ONNX Runtime.

```python
torch.onnx.export(
    model, 
    example_input, 
    "model.onnx",
    input_names=['input'],
    output_names=['output']
)
```

In [ ]:
torch.onnx.export(
    model, 
    example_input, 
    "simple_model.onnx",
    input_names=['input'],
    output_names=['output']
)
print("Exported to simple_model.onnx")

## 4. Quantization (Making it smaller)

We can convert weights from Float32 to Int8. This reduces model size by 4x and speeds up inference on supported hardware.

### Dynamic Quantization
Quantizes weights ahead of time, but activations are quantized dynamically at runtime. Good for LSTMs/Transformers.

In [ ]:
quantized_model = torch.quantization.quantize_dynamic(
    model, 
    {nn.Linear}, # Layers to quantize
    dtype=torch.qint8
)

print(f"Original Size: {os.path.getsize('traced_model.pt')/1024:.2f} KB")
# Note: For this tiny model, overhead might make it larger, but for large models it helps.
torch.jit.save(torch.jit.trace(quantized_model, example_input), "quantized_model.pt")
print(f"Quantized Size: {os.path.getsize('quantized_model.pt')/1024:.2f} KB")

## 5. Batch vs Online Inference (Critical Decision!)

Before deploying, you must choose between **batch** and **online** inference. This decision affects your entire architecture.

### Batch Inference (Offline)
- Generate predictions on a schedule (hourly, daily)
- Store results in a database for later retrieval
- **When to use**: Non-real-time applications, expensive models, large datasets
- **Examples**: Daily product recommendations, overnight credit scoring, monthly churn predictions

### Online Inference (Real-time)
- Generate predictions on-demand when requested
- Return results immediately (< 100ms typically)
- **When to use**: Real-time applications, personalization, user-facing features
- **Examples**: Fraud detection, chatbots, search ranking, content moderation

### Key Trade-offs

| Aspect | Batch Inference | Online Inference |
|--------|----------------|------------------|
| Latency | Hours to days | < 100ms |
| Cost | Lower (scheduled compute) | Higher (always-on servers) |
| Complexity | Simpler infrastructure | Requires load balancing, caching |
| Use Case | Periodic predictions | Real-time responses |
| Scalability | Easy (process in parallel) | Harder (handle traffic spikes) |

**Important**: Inference costs account for **up to 90% of production ML costs** in 2025!

In [ ]:
# Example: Batch Inference Pattern
import torch

# Simulate a batch of data
batch_data = torch.randn(1000, 10)  # 1000 samples

# Process in batches for efficiency
batch_size = 32
predictions = []

model.eval()
with torch.no_grad():
    for i in range(0, len(batch_data), batch_size):
        batch = batch_data[i:i+batch_size]
        preds = model(batch)
        predictions.append(preds)

# Combine all predictions
all_predictions = torch.cat(predictions, dim=0)
print(f"Batch inference: Processed {len(all_predictions)} samples")
print(f"Predictions shape: {all_predictions.shape}")

# In production, you would save these to a database:
# db.save_predictions(all_predictions)  # Store for later lookup

## 6. Modern Serving Frameworks for LLMs (2025 Update)

**IMPORTANT UPDATE**: TorchServe was archived in August 2025 and is now in "Limited Maintenance" mode. For production LLM deployment in 2025, use these modern alternatives:

### vLLM - The Current Standard
**Best for**: Flexibility, rapid deployment, Hugging Face integration

```python
# Installation: pip install vllm

from vllm import LLM, SamplingParams

# Initialize with advanced optimizations
llm = LLM(
    model="meta-llama/Llama-3-8B",
    tensor_parallel_size=2,           # Multi-GPU
    gpu_memory_utilization=0.95,      # Maximize GPU usage
    enable_prefix_caching=True,       # KV cache optimization
    max_model_len=8192               # Context length
)

# Automatic continuous batching happens under the hood
prompts = ["What is PyTorch?", "Explain neural networks"]
sampling_params = SamplingParams(temperature=0.7, max_tokens=100)

outputs = llm.generate(prompts, sampling_params)
for output in outputs:
    print(output.outputs[0].text)
```

**Key Features**:
- **PagedAttention**: Treats KV cache like OS virtual memory (huge memory savings!)
- **Continuous Batching**: No waiting for entire batch to finish
- **120-160 req/sec** typical throughput
- Hugging Face compatible out-of-the-box

### TensorRT-LLM - Maximum Performance
**Best for**: NVIDIA hardware, production systems requiring maximum throughput

**Key Features**:
- Custom NVIDIA kernels (fastest on H100/A100)
- **180-220 req/sec** throughput, **35-50ms** time-to-first-token
- FP8/INT4/INT8 quantization support
- **10,000+ tokens/sec** on H100 with FP8

**Trade-off**: Less flexible, NVIDIA-specific, harder setup

### When to Use Which?

| Framework | Use When | Pros | Cons |
|-----------|----------|------|------|
| **vLLM** | Standard LLM deployment | Easy, flexible, HF-compatible | Slightly slower than TensorRT |
| **TensorRT-LLM** | Max performance needed | Fastest, NVIDIA-optimized | Complex setup, vendor lock-in |
| **FastAPI + PyTorch** | Custom models, small scale | Full control, simple | Manual optimization needed |

## 7. Practical Example: FastAPI + PyTorch Serving

For custom models or when you need full control, FastAPI is the simplest production-ready option.

In [ ]:
# Save this as: serve_model.py
# Run with: uvicorn serve_model:app --reload

code_example = '''
from fastapi import FastAPI
from pydantic import BaseModel
import torch
import torch.nn as nn

app = FastAPI()

# Load model once at startup
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 50)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(50, 2)
    
    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

model = SimpleModel()
model.load_state_dict(torch.load("model_weights.pth"))
model.eval()

# Request/Response schema
class PredictionRequest(BaseModel):
    features: list[float]

class PredictionResponse(BaseModel):
    prediction: list[float]
    latency_ms: float

@app.post("/predict", response_model=PredictionResponse)
async def predict(request: PredictionRequest):
    import time
    start = time.time()
    
    # Convert to tensor
    x = torch.tensor([request.features])
    
    # Inference
    with torch.no_grad():
        output = model(x)
    
    latency = (time.time() - start) * 1000  # Convert to ms
    
    return PredictionResponse(
        prediction=output[0].tolist(),
        latency_ms=latency
    )

@app.get("/health")
async def health():
    return {"status": "healthy"}
'''

print("FastAPI Serving Example:")
print(code_example)
print("\nTo deploy:")
print("1. Save the code above as 'serve_model.py'")
print("2. Install: pip install fastapi uvicorn")
print("3. Run: uvicorn serve_model:app --host 0.0.0.0 --port 8000")
print("4. Test: curl -X POST http://localhost:8000/predict -H 'Content-Type: application/json' -d '{\"features\": [1,2,3,4,5,6,7,8,9,10]}'")


## Summary: 2025 Deployment Decision Tree

```
Start
  ↓
Are you deploying an LLM (Large Language Model)?
  ├─ Yes → Are you using NVIDIA GPUs (A100/H100)?
  │         ├─ Yes, need MAX performance → TensorRT-LLM
  │         └─ No, or need flexibility → vLLM
  │
  └─ No (Custom Model) → Do you need real-time predictions?
            ├─ Yes (< 100ms latency) → FastAPI + Docker + GPU
            └─ No (periodic predictions) → Batch Inference
                  ↓
            Do you need to deploy to production?
              ├─ Yes → Docker + Kubernetes + Monitoring
              └─ No → Local FastAPI server
```

## 2025 Production Checklist

**Export & Optimization:**
- ✅ TorchScript/ONNX for model export
- ✅ Quantization (INT8/INT4 for 4x smaller models)
- ✅ ONNX Runtime for edge deployment

**LLM Serving (if applicable):**
- ✅ vLLM for standard deployments (PagedAttention, continuous batching)
- ✅ TensorRT-LLM for maximum NVIDIA performance
- ⚠️  **AVOID**: TorchServe (archived August 2025, limited maintenance)

**Custom Model Serving:**
- ✅ FastAPI for RESTful APIs (lightweight, flexible)
- ✅ Docker for containerization (reproducibility)
- ✅ Kubernetes for orchestration (auto-scaling)

**Inference Pattern:**
- ✅ Online inference for real-time (< 100ms latency)
- ✅ Batch inference for periodic (lower cost, higher throughput)
- ⚠️  **Remember**: Inference = 90% of production ML costs!

**Monitoring (Critical!):**
- ✅ Latency metrics (P50, P95, P99)
- ✅ Throughput tracking (requests/sec)
- ✅ Model drift detection
- ✅ Error rate monitoring
- ✅ Resource utilization (GPU/CPU/Memory)

**Deployment Strategies:**
- ✅ Canary deployments (gradual rollout)
- ✅ Blue-Green deployments (zero downtime)
- ✅ A/B testing (compare model versions)

## What FAANG/Top Tech Companies Look For in 2025

- ✅ Can you choose the right inference pattern? (Batch vs Online)
- ✅ Can you optimize LLM inference? (vLLM, KV cache, quantization)
- ✅ Can you serve models efficiently? (FastAPI, modern frameworks)
- ✅ Can you containerize and orchestrate? (Docker, Kubernetes)
- ✅ Can you monitor production systems? (Latency, drift, errors)
- ✅ Can you handle deployments safely? (Canary, rollback, A/B testing)
- ✅ Do you understand cost optimization? (Inference is 90% of costs!)

You now know the **2025 state-of-the-art** for model deployment! 🚀